<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_6/02_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# モデルの訓練
対話文の生成するためのモデルを訓練し、保存します。

## ライブラリのインストール
分かち書きのためにjanomeを、テキストデータの前処理のためにtorchtextをインストールします。

In [ ]:
!pip install janome==0.4.1
!pip install torchvision==0.7.0
!pip install torchtext==0.7.0
!pip install torch==1.6.0

## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## データセットの読み込み
保存されている対話文のデータセットを読み込みます。

In [ ]:
import torch
import torchtext
import dill

path = "/content/drive/My Drive/live_ai_data/"  # 保存場所を指定

train_examples = torch.load(path+"train_examples.pkl", pickle_module=dill)
test_examples = torch.load(path+"test_examples.pkl", pickle_module=dill)

input_field = torch.load(path+"input_field.pkl", pickle_module=dill)
reply_field = torch.load(path+"reply_field.pkl", pickle_module=dill)

train_data = torchtext.data.Dataset(train_examples, [("inp_text", input_field), ("rep_text", reply_field)])
test_data = torchtext.data.Dataset(test_examples, [("inp_text", input_field), ("rep_text", reply_field)])

## Iteratorの設定
バッチごとに学習を行うために、Iteratorを設定します。  

In [ ]:
# Iteratorの設定
batch_size = 32

train_iterator = torchtext.data.Iterator(
    train_data,
    batch_size=batch_size, 
    train=True  # シャッフルして取り出す
)

test_iterator = torchtext.data.Iterator(
    test_data,
    batch_size=batch_size, 
    train=False,
    sort=False
)

ミニバッチを取り出して、内容を表示します。  
ミニバッチには、単語をインデックスに置き換えた文章が格納されます。



In [ ]:
batch = next(iter(train_iterator))  # ミニバッチを取り出す
print(batch.inp_text.size())  # ミニバッチにおける入力のサイズ
print(batch.inp_text[0])  # 最初の要素
print(batch.rep_text.size())  # ミニバッチにおける応答のサイズ
print(batch.rep_text[0])  # 最初の要素

## Encoderのクラス
Encoderをクラスとして実装します。  
RNN部分にはGRUを使用しますが、双方向に情報が流れるBidirectional RNNを設定可能にします。  
Bidirectional RNNが設定されている場合、Encoderの出力と隠れ層の値は双方向の時間の値を足したものになります。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, n_h, n_vocab, n_emb, num_layers=1, bidirectional=False, dropout=0.0):
        super().__init__()
        
        self.n_h = n_h
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = dropout  # ドロップアウト層

        # 埋め込み層
        self.embedding = nn.Embedding(n_vocab, n_emb)
        self.embedding_dropout = nn.Dropout(self.dropout)

        self.gru = nn.GRU(  # GRU層
            input_size=n_emb,  # 入力サイズ
            hidden_size=n_h,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
            num_layers=num_layers,  # RNN層の数（層を重ねることも可能）
            bidirectional=bidirectional,  # Bidrectional RNN
        )

    def forward(self, x):
        # 文章の長さを取得
        idx_pad = input_field.vocab.stoi["<pad>"]
        sentence_lengths = x.size()[1] - (x == idx_pad).sum(dim=1)

        y = self.embedding(x)  # 単語をベクトルに変換
        y = self.embedding_dropout(y)
        y = nn.utils.rnn.pack_padded_sequence(  # 入力のパッキング
            y,
            sentence_lengths,
            batch_first=True,
            enforce_sorted=False
            )
        y, h = self.gru(y)

        y, _ = nn.utils.rnn.pad_packed_sequence(y, batch_first=True)  # テンソルに戻す
        if self.bidirectional:  # 双方向の値を足し合わせる
            y = y[:, :, :self.n_h] + y[:, :, self.n_h:]
            h = h[:self.num_layers] + h[self.num_layers:]
        return y, h

## Decoderのクラス
Decoderをクラスとして実装します。  
RNN部分にはGRUを使用します。  
GRU層の出力は、全結合層を経てDecoderの出力となります。

In [ ]:
class Decoder(nn.Module):
    def __init__(self, n_h, n_out, n_vocab, n_emb, num_layers=1, dropout=0.0):
        super().__init__()
        
        self.n_h = n_h
        self.n_out = n_out
        self.num_layers = num_layers
        self.dropout = dropout

        # 埋め込み層
        self.embedding = nn.Embedding(n_vocab, n_emb)
        self.embedding_dropout = nn.Dropout(self.dropout)  # ドロップアウト層

        self.gru = nn.GRU(  # GRU層
            input_size=n_emb,  # 入力サイズ
            hidden_size=n_h,  # ニューロン数
            batch_first=True,  # 入力を (バッチサイズ, 時系列の数, 入力の数) にする
            num_layers=num_layers,  # RNN層の数（層を重ねることも可能）
        )

        self.fc = nn.Linear(n_h*2, self.n_out)  # コンテキストベクトルが合流するので2倍のサイズ
                
    def forward(self, x, h_encoder, y_encoder):
        y = self.embedding(x)  # 単語をベクトルに変換
        y = self.embedding_dropout(y)
        y, h = self.gru(y, h_encoder)

        # ----- Attension -----
        y_tr = torch.transpose(y, 1, 2)  # 次元1と次元2を入れ替える
        ed_mat = torch.bmm(y_encoder, y_tr)  # バッチごとに行列積
        attn_weight = F.softmax(ed_mat, dim=1)  # attension weightの計算
        attn_weight_tr = torch.transpose(attn_weight, 1, 2)  # 次元1と次元2を入れ替える
        context = torch.bmm(attn_weight_tr, y_encoder)  # コンテキストベクトルの計算
        y = torch.cat([y, context], dim=2)  # 出力とコンテキストベクトルの合流

        y = self.fc(y)
        y = F.softmax(y, dim=2)
        
        return y, h

## Seq2Seqのクラス
Seq2Seqを構築します。  
`is_gpu`が`True`であれば、GPU対応を行います。

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, is_gpu=True):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.is_gpu = is_gpu
        if self.is_gpu:
            self.encoder.cuda()
            self.decoder.cuda()
        
    def forward(self, x_encoder, x_decoder):  # 訓練に使用
        if self.is_gpu:
            x_encoder, x_decoder = x_encoder.cuda(), x_decoder.cuda()

        batch_size = x_decoder.shape[0]
        n_time = x_decoder.shape[1]
        y_encoder, h = self.encoder(x_encoder)

        y_decoder = torch.zeros(batch_size, n_time, self.decoder.n_out)
        if self.is_gpu:
            y_decoder = y_decoder.cuda()

        for t in range(0, n_time):
            x = x_decoder[:, t:t+1]  # Decoderの入力を使用
            y, h= self.decoder(x, h, y_encoder)
            y_decoder[:, t:t+1, :] = y
        return y_decoder

    def predict(self, x_encoder):  # 予測に使用
        if self.is_gpu:
            x_encoder = x_encoder.cuda()

        batch_size = x_encoder.shape[0]
        n_time = x_encoder.shape[1]
        y_encoder, h = self.encoder(x_encoder)

        y_decoder = torch.zeros(batch_size, n_time, dtype=torch.long)
        if self.is_gpu:
            y_decoder = y_decoder.cuda() 

        y = torch.ones(batch_size, 1, dtype=torch.long) * input_field.vocab.stoi["<sos>"]
        for t in range(0, n_time):
            x = y  # 前の時刻の出力を入力に
            if self.is_gpu:
                x = x.cuda()
            y, h= self.decoder(x, h, y_encoder)
            y = y.argmax(2)
            y_decoder[:, t:t+1] = y  
        return y_decoder

## 評価用の関数
実際に入力文に対して返答を生成し、モデルを評価するための関数を定義します。

In [ ]:
def evaluate_model(model, iterator):
    model.eval()  # 評価モード

    batch = next(iter(iterator))
    x = batch.inp_text
    y = model.predict(x)
    for i in range(x.size()[0]):
        inp_text = ""
        for j in range(x.size()[1]):
            word = input_field.vocab.itos[x[i][j]]
            if word=="<pad>":
                break
            inp_text += word

        rep_text = ""
        for j in range(y.size()[1]):
            word = reply_field.vocab.itos[y[i][j]]
            if word=="<eos>":
                break
            rep_text += word

        print("input:", inp_text)
        print("reply:", rep_text)
        print()

## 学習
DataLoaderを使ってミニバッチを取り出し、Seq2Seqのモデルを訓練します。  
今回は、評価用データの誤差の減少が確認できなくなった時点で訓練を終了します（早期終了）。


In [ ]:
from torch import optim

is_gpu = True  # GPUを使用するかどうか
n_h = 896
n_vocab_inp = len(input_field.vocab.itos)
n_vocab_rep = len(reply_field.vocab.itos)
n_emb = 300
n_out = n_vocab_rep
early_stop_patience = 5  # 早期終了のタイミング（誤差の最小値が何回更新されなかったら終了か）
num_layers = 1
bidirectional = True
dropout = 0.0
clip = 100

# Seq2Seqのモデルを構築
encoder = Encoder(n_h, n_vocab_inp, n_emb, num_layers, bidirectional, dropout=dropout)
decoder = Decoder(n_h, n_out, n_vocab_rep, n_emb, num_layers, dropout=dropout)
seq2seq = Seq2Seq(encoder, decoder, is_gpu=is_gpu)

# 誤差関数
loss_fnc = nn.CrossEntropyLoss(ignore_index=reply_field.vocab.stoi["<pad>"])

# 最適化アルゴリズム
optimizer_enc = optim.Adam(seq2seq.parameters(), lr=0.0001)
optimizer_dec = optim.Adam(seq2seq.parameters(), lr=0.0005)

# 損失のログ
record_loss_train = []
record_loss_test = []
min_losss_test = 0.0

# 学習
for i in range(1000):
    # 訓練モード
    seq2seq.train()

    loss_train = 0
    for j, batch in enumerate(train_iterator):
        inp, rep = batch.inp_text, batch.rep_text
        x_enc = inp
        x_dec = rep[:, :-1]
        y_dec = seq2seq(x_enc, x_dec)

        t_dec = rep[:, 1:]
        t_dec = t_dec.cuda() if is_gpu else t_dec
        loss = loss_fnc(
            y_dec.view(-1, y_dec.size()[2]),
            t_dec.view(-1)
            )
        loss_train += loss.item()
        optimizer_enc.zero_grad()
        optimizer_dec.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(encoder.parameters(), clip)
        nn.utils.clip_grad_norm_(decoder.parameters(), clip)
        optimizer_enc.step()
        optimizer_dec.step()

        if j%1000==0:
            print("batch:", str(j)+"/"+str(len(train_data)//batch_size+1), "loss:", loss.item())
    loss_train /= j+1
    record_loss_train.append(loss_train)

    # 評価モード
    seq2seq.eval()

    loss_test = 0
    for j, batch in enumerate(test_iterator):
        inp, rep = batch.inp_text, batch.rep_text
        x_enc = inp
        x_dec = torch.ones(rep.size(), dtype=torch.long) * reply_field.vocab.stoi["<sos>"]
        x_dec[:, 1:] = rep[:, :-1]
        y_dec = seq2seq(x_enc, x_dec)

        t_dec = rep.cuda() if is_gpu else rep
        loss = loss_fnc(
            y_dec.view(-1, y_dec.size()[2]),
            t_dec.view(-1)
            )
        loss_test += loss.item()
    loss_test /= j+1
    record_loss_test.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)
        print()

    evaluate_model(seq2seq, test_iterator)

    # ----- 早期終了 -----
    latest_min = min(record_loss_test[-(early_stop_patience):])  # 直近の最小値
    if len(record_loss_test) >= early_stop_patience:
        if latest_min > min_loss_test:  # 直近で最小値が更新されていなければ
            print("Early stopping!")
            break
        min_loss_test = latest_min
    else:
        min_loss_test = latest_min

## 誤差の推移
誤差の推移をグラフ表示します。  

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## モデルの保存
訓練済みモデルのパラメータを保存します。    
`state_dict()`によりモデルの各パラメータが取得できるので、これを保存します。

In [ ]:
import torch

# state_dict()の表示
for key in seq2seq.state_dict():
    print(key, ":", seq2seq.state_dict()[key].size())
print(seq2seq.state_dict()["encoder.gru.weight_ih_l0"][0])  # 　パラメータの一部を表示

# 保存
torch.save(seq2seq.state_dict(), path+"model_bot.pth")  

## モデルの読み込み
保存したパラメータを読み込み、モデルに設定します。  
`torch.load()`で`map_location`にCPUを指定することで、GPUで訓練したモデルをCPUで使用することが可能になります。  


In [ ]:
# 読み込み
encoder_loaded = Encoder(n_h, n_vocab_inp, n_emb, num_layers, bidirectional, dropout=dropout)
decoder_loaded = Decoder(n_h, n_out, n_vocab_rep, n_emb, num_layers, dropout=dropout)
seq2seq_loaded = Seq2Seq(encoder_loaded, decoder_loaded, is_gpu=is_gpu)

seq2seq_loaded.load_state_dict(torch.load(path+"model_bot.pth", map_location=torch.device("cpu")))  #CPU対応
seq2seq_loaded.eval()  # 評価モード

# state_dict()の表示
for key in seq2seq_loaded.state_dict():
    print(key, ": ", seq2seq_loaded.state_dict()[key].size())
print(seq2seq_loaded.state_dict()["encoder.gru.weight_ih_l0"][0])  # 　パラメータの一部を表示

モデルの各パラメータを保存し、読み込むことができました。